# SpecEIS Sediment Model - Equation Reference
## Human-Readable Mathematical Formulation

This notebook extracts and displays all mathematical equations from the SpecEIS glacier-sediment model in textbook-style LaTeX formatting for easy reference and understanding.

**Model Overview:**
- Coupled ice sheet dynamics with sediment transport
- Includes bedrock erosion, fluvial sediment transport, and sediment diffusion
- Firedrake-based finite element solver
- 1D depth-averaged model with vertical velocity profile parameterization

In [28]:
import numpy as np
import re
from pathlib import Path
from IPython.display import display, Markdown, Latex
import sympy as sp

# Load the model script
model_path = Path("/home/kayatroyer/Repositories/brinkmodeltest/ModelScripts/SpecEIS-sediment_newmodel")
with open(model_path, 'r') as f:
    model_script = f.read()

print("✓ Model script loaded successfully")
print(f"Script length: {len(model_script)} characters")

✓ Model script loaded successfully
Script length: 42902 characters


## Section 1: Model Parameters and Physical Constants

These are the fundamental parameters that define the model's physical regime.

In [29]:
# Extract and display parameters
parameters = {
    "Domain": {
        "L": ("55000", "m", "Characteristic domain length"),
        "zmin": ("-300.0", "m", "Minimum elevation"),
        "zmax": ("2200.0", "m", "Maximum elevation"),
        "thklim": ("1.0", "m", "Minimum ice thickness threshold"),
    },
    "Material Properties": {
        r"\rho_i": ("917", r"kg\,m^{-3}", "Ice density"),
        r"\rho_w": ("1029", r"kg\,m^{-3}", "Seawater density"),
        r"\rho_s": ("1600", r"kg\,m^{-3}", "Sediment density"),
        r"\rho_r": ("2650", r"kg\,m^{-3}", "Bedrock density"),
    },
    "Flow Parameters": {
        "g": ("9.81", r"m\,s^{-2}", "Gravitational acceleration"),
        "n": ("3.0", "-", "Glen's exponent"),
        "m": ("1.0", "-", "Sliding law exponent"),
        r"b": (r"1 \times 10^{-16(-1/n)}", r"Pa^{-3}\,yr", "Ice hardness parameter"),
        r"\epsilon_{\text{reg}}": (r"10^{-4}", "-", "Regularization parameter"),
    },
    "Thermal": {
        "La": (r"3.35 \times 10^5", r"J\,kg^{-1}", "Latent heat of melting"),
    },
    "Sediment Process Coefficients": {
        r"l_s": ("2.0", "m", "Reference sediment thickness for erosion limiting"),
        r"b_e": (r"2 \times 10^{-9}", r"yr^{-1}", "Bedrock erosion coefficient"),
        r"c_c": (r"5 \times 10^{-11}", r"yr^{-1}", "Fluvial erosion coefficient"),
        "d": ("500", "-", "Fallout fraction for suspended load"),
        r"h_0": ("0.1", "m", "Subglacial cavity depth parameter"),
        r"k_{\text{diff}}": ("20", r"m^2\,yr^{-1}", "Sediment diffusivity"),
        r"h_{\text{ref}}": ("10", "m", "Reference sediment thickness for diffusion"),
    },
    "Water Pressure Coupling": {
        "k": ("0.7", "-", "Fraction of overburden defining water pressure"),
        r"\beta_0": ("50", r"m^{-1}", "Basal friction coefficient"),
    },
    "Shelf Dynamics": {
        r"\text{scf}": ("0.2", r"yr^{-1}", "Shelf collapse factor"),
    },
    "Temporal": {
        r"\Delta t": ("0.1", "yr", "Initial time step"),
        r"\Delta t_{\max}": ("1.0", "yr", "Maximum time step"),
        r"t_{\text{end}}": ("12950", "yr", "Total simulation time"),
    }
}

# Create LaTeX table
latex_params = r"\begin{array}{l c c l}" + "\n"
latex_params += r"\textbf{Parameter} & \textbf{Value} & \textbf{Units} & \textbf{Description} \\" + "\n"

for category, params in parameters.items():
    for var, (val, units, desc) in params.items():
        latex_params += f"{var} & {val} & {units} & \\text{{{desc}}} \\\\" + "\n"

latex_params = latex_params.rstrip() + r"\end{array}"

display(Markdown(f"### Parameter Table\n$$\n{latex_params}\n$$"))

### Parameter Table
$$
\begin{array}{l c c l}
\textbf{Parameter} & \textbf{Value} & \textbf{Units} & \textbf{Description} \\
L & 55000 & m & \text{Characteristic domain length} \\
zmin & -300.0 & m & \text{Minimum elevation} \\
zmax & 2200.0 & m & \text{Maximum elevation} \\
thklim & 1.0 & m & \text{Minimum ice thickness threshold} \\
\rho_i & 917 & kg\,m^{-3} & \text{Ice density} \\
\rho_w & 1029 & kg\,m^{-3} & \text{Seawater density} \\
\rho_s & 1600 & kg\,m^{-3} & \text{Sediment density} \\
\rho_r & 2650 & kg\,m^{-3} & \text{Bedrock density} \\
g & 9.81 & m\,s^{-2} & \text{Gravitational acceleration} \\
n & 3.0 & - & \text{Glen's exponent} \\
m & 1.0 & - & \text{Sliding law exponent} \\
b & 1 \times 10^{-16(-1/n)} & Pa^{-3}\,yr & \text{Ice hardness parameter} \\
\epsilon_{\text{reg}} & 10^{-4} & - & \text{Regularization parameter} \\
La & 3.35 \times 10^5 & J\,kg^{-1} & \text{Latent heat of melting} \\
l_s & 2.0 & m & \text{Reference sediment thickness for erosion limiting} \\
b_e & 2 \times 10^{-9} & yr^{-1} & \text{Bedrock erosion coefficient} \\
c_c & 5 \times 10^{-11} & yr^{-1} & \text{Fluvial erosion coefficient} \\
d & 500 & - & \text{Fallout fraction for suspended load} \\
h_0 & 0.1 & m & \text{Subglacial cavity depth parameter} \\
k_{\text{diff}} & 20 & m^2\,yr^{-1} & \text{Sediment diffusivity} \\
h_{\text{ref}} & 10 & m & \text{Reference sediment thickness for diffusion} \\
k & 0.7 & - & \text{Fraction of overburden defining water pressure} \\
\beta_0 & 50 & m^{-1} & \text{Basal friction coefficient} \\
\text{scf} & 0.2 & yr^{-1} & \text{Shelf collapse factor} \\
\Delta t & 0.1 & yr & \text{Initial time step} \\
\Delta t_{\max} & 1.0 & yr & \text{Maximum time step} \\
t_{\text{end}} & 12950 & yr & \text{Total simulation time} \\\end{array}
$$

## Section 2: Geometric and Derived Quantities

These variables are derived from the primary solution variables and define the glacier geometry.

In [30]:
# Geometric definitions
geometric_eqs = [
    (r"\{Sediment surface elevation}", r"B_{\text{hat}} = B + h_s", 
     "Top of sediment layer; combines bedrock and sediment thickness"),
    
    (r"{Water surface}", r"\ell = \text{softplus}(0, B_{\text{hat}}, \alpha=5)", 
     "Water surface is the maximum of zero and sediment surface"),
    
    (r"{Ice base elevation}", r"S_{\text{base}} = \text{softplus}(B_{\text{hat}}, -\frac{\rho_i}{\rho_w}H, \alpha=5)", 
     "Greater of sediment surface or floating ice base; includes hydrostatic balance"),
    
    (r"{Water depth}", r"D = \text{softplus}(-B_{\text{hat}}, 0, \alpha=5)", 
     "Depth of water above sea level"),
    
    (r"{Ice surface elevation}", r"S = S_{\text{base}} + H", 
     "Sum of ice base elevation and ice thickness"),
]

display(Markdown("### Geometric Relationships\n"))

for name, eq, desc in geometric_eqs:
    latex_str = f"**{name}:**\n\n$$\n{eq}\n$$\n\n_{desc}_\n"
    display(Markdown(latex_str))

# Define softplus function
display(Markdown("""
### Smoothing Function: Softplus

The **softplus** function is a differentiable approximation to the ramp function $\\max(a,b)$:

$$\\text{softplus}(a, b, \\alpha) = \\max(a,b) + \\frac{1}{\\alpha}\\ln\\left(1 + e^{\\alpha(\\min(a,b)-\\max(a,b))}\\right)$$

**Properties:**
- Smoothly transitions between $a$ and $b$ 
- Parameter $\\alpha$ controls sharpness (larger $\\alpha$ → sharper transition)
- Maintains differentiability for finite element solver
- Used for: water surface (avoiding sharp breaks), ice base position, water depth
"""))

### Geometric Relationships


**\{Sediment surface elevation}:**

$$
B_{\text{hat}} = B + h_s
$$

_Top of sediment layer; combines bedrock and sediment thickness_


**{Water surface}:**

$$
\ell = \text{softplus}(0, B_{\text{hat}}, \alpha=5)
$$

_Water surface is the maximum of zero and sediment surface_


**{Ice base elevation}:**

$$
S_{\text{base}} = \text{softplus}(B_{\text{hat}}, -\frac{\rho_i}{\rho_w}H, \alpha=5)
$$

_Greater of sediment surface or floating ice base; includes hydrostatic balance_


**{Water depth}:**

$$
D = \text{softplus}(-B_{\text{hat}}, 0, \alpha=5)
$$

_Depth of water above sea level_


**{Ice surface elevation}:**

$$
S = S_{\text{base}} + H
$$

_Sum of ice base elevation and ice thickness_



### Smoothing Function: Softplus

The **softplus** function is a differentiable approximation to the ramp function $\max(a,b)$:

$$\text{softplus}(a, b, \alpha) = \max(a,b) + \frac{1}{\alpha}\ln\left(1 + e^{\alpha(\min(a,b)-\max(a,b))}\right)$$

**Properties:**
- Smoothly transitions between $a$ and $b$ 
- Parameter $\alpha$ controls sharpness (larger $\alpha$ → sharper transition)
- Maintains differentiability for finite element solver
- Used for: water surface (avoiding sharp breaks), ice base position, water depth


## Section 3: Velocity Profile Parameterization

The depth-averaged model uses a vertical basis function expansion to represent velocity variation with depth.

In [31]:
display(Markdown(r"""
### Vertical Velocity Structure

The velocity at depth $s$ (normalized vertical coordinate, $s \in [0,1]$) is represented as:

$$u(s) = \bar{u} \cdot \phi_1(s) + u_{\text{def}} \cdot \phi_2(s)$$

**Basis Functions:**

$$\phi_1(s) = 1 \quad \text{(depth-averaged component)}$$

$$\phi_2(s) = \frac{1}{p}\left[(p+1)s^p - 1\right], \quad p = 4$$

**Vertical Derivatives (for stress calculations):**

$$\frac{d\phi_1}{ds} = 0$$

$$\frac{d\phi_2}{ds} = (p+1)s^{p-1}$$

**Normalized Vertical Coordinate:**

$$s \in [0,1] \quad \text{where} \quad s=0 \text{ at bed}, \quad s=1 \text{ at surface}$$

$$\frac{ds}{dz} = -\frac{1}{H}$$

**Model Components:**
- $\bar{u}$: Depth-averaged (vertically-integrated) velocity
- $u_{\text{def}}$: Deformational velocity component (accounts for nonlinear flow)
- $u_{\text{surface}}(1) = \bar{u} + u_{\text{def}}$: Surface velocity
- $u_{\text{basal}}(0) = \bar{u}$: Basal velocity (includes sliding)
"""))


### Vertical Velocity Structure

The velocity at depth $s$ (normalized vertical coordinate, $s \in [0,1]$) is represented as:

$$u(s) = \bar{u} \cdot \phi_1(s) + u_{\text{def}} \cdot \phi_2(s)$$

**Basis Functions:**

$$\phi_1(s) = 1 \quad \text{(depth-averaged component)}$$

$$\phi_2(s) = \frac{1}{p}\left[(p+1)s^p - 1\right], \quad p = 4$$

**Vertical Derivatives (for stress calculations):**

$$\frac{d\phi_1}{ds} = 0$$

$$\frac{d\phi_2}{ds} = (p+1)s^{p-1}$$

**Normalized Vertical Coordinate:**

$$s \in [0,1] \quad \text{where} \quad s=0 \text{ at bed}, \quad s=1 \text{ at surface}$$

$$\frac{ds}{dz} = -\frac{1}{H}$$

**Model Components:**
- $\bar{u}$: Depth-averaged (vertically-integrated) velocity
- $u_{\text{def}}$: Deformational velocity component (accounts for nonlinear flow)
- $u_{\text{surface}}(1) = \bar{u} + u_{\text{def}}$: Surface velocity
- $u_{\text{basal}}(0) = \bar{u}$: Basal velocity (includes sliding)


## Section 4: Ice Flow Constitutive Relations

These equations define how stress relates to strain rate in ice.

In [32]:
display(Markdown(r"""
### Glen's Flow Law and Effective Viscosity

**Glen's Flow Law (constitutive relation for ice):**

$$\dot{\varepsilon} = A \tau^n$$

where:
- $\dot{\varepsilon}$ is strain rate
- $\tau$ is deviatoric stress
- $n = 3$ is Glen's exponent
- $A(T)$ is the ice hardness parameter (temperature-dependent)

**Effective Viscosity Formulation:**

$$\eta_v(s) = \frac{b}{2}\left[\left(\frac{\partial u}{\partial x}\bigg|_s + \frac{\partial u}{\partial s}\frac{ds}{dx}\right)^2 + \frac{1}{4}\left(\frac{\partial u}{\partial s}\frac{ds}{dz}\right)^2 + \epsilon_{\text{reg}}\right]^{\frac{1-n}{2n}}$$

**Components:**

$$\frac{\partial u}{\partial x}\bigg|_s = \sum_i \frac{\partial u_i}{\partial x}\phi_i(s) \quad \text{(horizontal strain rate)}$$

$$\frac{\partial u}{\partial s} = \sum_i u_i \frac{d\phi_i}{ds} \quad \text{(vertical strain rate)}$$

**Physical Interpretation:**
- Effective viscosity increases with slower deformation (lower strain rates)
- Decreases with higher strain rates (shear thinning)
- Regularization parameter $\epsilon_{\text{reg}} = 10^{-4}$ prevents singularities at zero strain rate
"""))


### Glen's Flow Law and Effective Viscosity

**Glen's Flow Law (constitutive relation for ice):**

$$\dot{\varepsilon} = A \tau^n$$

where:
- $\dot{\varepsilon}$ is strain rate
- $\tau$ is deviatoric stress
- $n = 3$ is Glen's exponent
- $A(T)$ is the ice hardness parameter (temperature-dependent)

**Effective Viscosity Formulation:**

$$\eta_v(s) = \frac{b}{2}\left[\left(\frac{\partial u}{\partial x}\bigg|_s + \frac{\partial u}{\partial s}\frac{ds}{dx}\right)^2 + \frac{1}{4}\left(\frac{\partial u}{\partial s}\frac{ds}{dz}\right)^2 + \epsilon_{\text{reg}}\right]^{\frac{1-n}{2n}}$$

**Components:**

$$\frac{\partial u}{\partial x}\bigg|_s = \sum_i \frac{\partial u_i}{\partial x}\phi_i(s) \quad \text{(horizontal strain rate)}$$

$$\frac{\partial u}{\partial s} = \sum_i u_i \frac{d\phi_i}{ds} \quad \text{(vertical strain rate)}$$

**Physical Interpretation:**
- Effective viscosity increases with slower deformation (lower strain rates)
- Decreases with higher strain rates (shear thinning)
- Regularization parameter $\epsilon_{\text{reg}} = 10^{-4}$ prevents singularities at zero strain rate


## Section 5: Stress Components

These define the longitudinal and shear stresses used in the momentum balance.

In [33]:
display(Markdown(r"""
### Stress Tensor Components

**Longitudinal (extensional) stress:**

$$\sigma_{xx}(s) = 4\eta_v(s)H\left(\frac{\partial u}{\partial x}\bigg|_s + \frac{\partial u}{\partial s}\frac{ds}{dx}\right)$$

**Vertical shear stress:**

$$\tau_{xz}(s) = \eta_v(s)H\left(\frac{ds}{dz}\right)^2 \frac{\partial u}{\partial s}$$

**Membrane stress (integrated over depth):**

$$\sigma_{xx}^{\text{mem}} = \int_0^1 \sigma_{xx}(s) \, ds$$

**Basal shear stress:**

$$\tau_b = \beta^2 u_{\text{basal}} \cdot N$$

where $N = \max(P_0 - P_w, 0)$ is the effective pressure.

**Driving stress:**

$$\tau_d = -\rho g H \frac{\partial S}{\partial x}$$

**Force balance interpretation:**
- Longitudinal stress gradients transmit stress along glacier
- Shear stress couples to basal motion through friction law
- Driving stress from ice surface slope provides primary forcing
"""))


### Stress Tensor Components

**Longitudinal (extensional) stress:**

$$\sigma_{xx}(s) = 4\eta_v(s)H\left(\frac{\partial u}{\partial x}\bigg|_s + \frac{\partial u}{\partial s}\frac{ds}{dx}\right)$$

**Vertical shear stress:**

$$\tau_{xz}(s) = \eta_v(s)H\left(\frac{ds}{dz}\right)^2 \frac{\partial u}{\partial s}$$

**Membrane stress (integrated over depth):**

$$\sigma_{xx}^{\text{mem}} = \int_0^1 \sigma_{xx}(s) \, ds$$

**Basal shear stress:**

$$\tau_b = \beta^2 u_{\text{basal}} \cdot N$$

where $N = \max(P_0 - P_w, 0)$ is the effective pressure.

**Driving stress:**

$$\tau_d = -\rho g H \frac{\partial S}{\partial x}$$

**Force balance interpretation:**
- Longitudinal stress gradients transmit stress along glacier
- Shear stress couples to basal motion through friction law
- Driving stress from ice surface slope provides primary forcing


## Section 6: Pressure and Basal Friction

These equations relate water pressure to ice overburden and define the sliding law.

In [46]:
display(Markdown(r"""
### Pressure Relationships

**Overburden pressure (lithostatic):**

$$P_0 = \rho_i g H$$

**Water pressure (from subglacial drainage):**

$$P_w = \max\left(kP_0, \frac{\rho_w}{\rho_i}(\ell - S_{\text{base}})\right)$$

**Effective pressure (reduced by water pressure):**

$$N = \max(P_0 - P_w, 0)$$

**Physical Interpretation:**
- Coupling parameter $k = 0.7$ relates water pressure to ice overburden
- When $k=0$: no water pressure effects (dry bed)
- When $k=1$: full coupling to overburden (wet bed)
- Alternative: hydrostatic balance if water fills cavity above flotation

### Basal Friction Law

**Weertman-type sliding law:**

$$\tau_b = \beta^2 u_{\text{basal}} \cdot N^m$$

where:
- $\beta^2 = 50$ m$^{-1}$ is the friction coefficient
- $u_{\text{basal}} = \bar{u}$ is depth-averaged velocity
- $m = 1$ is the sliding law exponent

**Weak-form representation:**
The momentum balance includes the term:

$$-\beta^2 N u(1) \quad \text{(at base)}$$

where $u(1)$ includes both sliding and deformation.
"""))


### Pressure Relationships

**Overburden pressure (lithostatic):**

$$P_0 = \rho_i g H$$

**Water pressure (from subglacial drainage):**

$$P_w = \max\left(kP_0, \frac{\rho_w}{\rho_i}(\ell - S_{\text{base}})\right)$$

**Effective pressure (reduced by water pressure):**

$$N = \max(P_0 - P_w, 0)$$

**Physical Interpretation:**
- Coupling parameter $k = 0.7$ relates water pressure to ice overburden
- When $k=0$: no water pressure effects (dry bed)
- When $k=1$: full coupling to overburden (wet bed)
- Alternative: hydrostatic balance if water fills cavity above flotation

### Basal Friction Law

**Weertman-type sliding law:**

$$\tau_b = \beta^2 u_{\text{basal}} \cdot N^m$$

where:
- $\beta^2 = 50$ m$^{-1}$ is the friction coefficient
- $u_{\text{basal}} = \bar{u}$ is depth-averaged velocity
- $m = 1$ is the sliding law exponent

**Weak-form representation:**
The momentum balance includes the term:

$$-\beta^2 N u(1) \quad \text{(at base)}$$

where $u(1)$ includes both sliding and deformation.


## Section 7: Mass Balance Equation

Conservation of ice mass drives thickness evolution.

In [35]:
display(Markdown(r"""
### Ice Mass Balance

**Strong form (conservation statement):**

$$\frac{\partial H}{\partial t} + \nabla \cdot (H\mathbf{u}) = \dot{a} - \text{scf} \cdot H \cdot \mathcal{F}$$

where:
- $H$ is ice thickness
- $\mathbf{u}$ is the horizontal velocity field
- $\dot{a}$ is surface mass balance (accumulation rate)
- $\text{scf}$ is the shelf collapse factor
- $\mathcal{F}$ is the flotation indicator

**Surface Mass Balance (SMB):**

$$\dot{a}(x) = a_{\max} + (a_{\min} - a_{\max})\frac{x}{2L}$$

$$a_{\min} = -16 \text{ m/yr}, \quad a_{\max} = 8 \text{ m/yr}$$

**Flotation Indicator:**

$$\mathcal{F} = \begin{cases} 1 & \text{if } H < -\frac{\rho_i}{\rho_w}B_{\text{hat}} \text{ (floating)} \\ 0 & \text{otherwise (grounded)} \end{cases}$$

**Shelf Collapse Model:**
- Removes fraction scf = 0.2 of floating ice per year
- Represents calving/fragmentation at shelf front

### Weak Form (Discontinuous Galerkin)

$$\int_\Omega \xi\frac{H-H_0}{\Delta t} dx + \int_\Omega \xi\left(\dot{a} - \text{scf} \cdot H \cdot \mathcal{F}\right) dx + \int_{\partial\Omega} \mathbf{u}\cdot\mathbf{n}\,H\,\xi \, ds = \text{flux corrections}$$

where $\xi$ is the test function.
"""))


### Ice Mass Balance

**Strong form (conservation statement):**

$$\frac{\partial H}{\partial t} + \nabla \cdot (H\mathbf{u}) = \dot{a} - \text{scf} \cdot H \cdot \mathcal{F}$$

where:
- $H$ is ice thickness
- $\mathbf{u}$ is the horizontal velocity field
- $\dot{a}$ is surface mass balance (accumulation rate)
- $\text{scf}$ is the shelf collapse factor
- $\mathcal{F}$ is the flotation indicator

**Surface Mass Balance (SMB):**

$$\dot{a}(x) = a_{\max} + (a_{\min} - a_{\max})\frac{x}{2L}$$

$$a_{\min} = -16 \text{ m/yr}, \quad a_{\max} = 8 \text{ m/yr}$$

**Flotation Indicator:**

$$\mathcal{F} = \begin{cases} 1 & \text{if } H < -\frac{\rho_i}{\rho_w}B_{\text{hat}} \text{ (floating)} \\ 0 & \text{otherwise (grounded)} \end{cases}$$

**Shelf Collapse Model:**
- Removes fraction scf = 0.2 of floating ice per year
- Represents calving/fragmentation at shelf front

### Weak Form (Discontinuous Galerkin)

$$\int_\Omega \xi\frac{H-H_0}{\Delta t} dx + \int_\Omega \xi\left(\dot{a} - \text{scf} \cdot H \cdot \mathcal{F}\right) dx + \int_{\partial\Omega} \mathbf{u}\cdot\mathbf{n}\,H\,\xi \, ds = \text{flux corrections}$$

where $\xi$ is the test function.


## Section 8: Momentum Balance (Stress Balance)

The force balance equation governs velocity evolution.

In [36]:
display(Markdown(r"""
### Momentum Balance Equation

**Strong form (force balance):**

$$-\nabla \cdot \boldsymbol{\sigma} - \rho_i g \nabla S = -\beta^2 N u_{\text{basal}} \quad \text{(at base)}$$

**Weak Form:**

The variational form integrates stress divergence over the domain:

$$\int_\Omega \left[-\frac{\partial \sigma_{xx}}{\partial x} - \rho g H \frac{\partial S}{\partial x}\right]\phi \, dx + \text{(stress boundary terms)} = \int_\Omega \beta^2 N u \phi \, dx$$

### Integrated Stress Components

Using the vertical basis function expansion, stresses are integrated:

$$\sigma_{xx}^{\text{integrated}} = \int_0^1 4\eta_v(s)H\left(\frac{\partial u}{\partial x}\bigg|_s\right) ds$$

$$\tau_{xz}^{\text{integrated}} = \int_0^1 \eta_v(s)H \left(\frac{ds}{dz}\right)^2 \frac{\partial u}{\partial s} ds$$

### Driving Stress (Nonstandard Formulation)

$$-\int_\Omega \frac{\partial(\rho g H \phi_{\bar{u}})}{\partial x}S \, dx + \text{(discontinuity corrections at element interfaces)} + \text{(boundary flux)}$$

**Physical Interpretation:**
- Momentum is transmitted through internal stresses
- Basal traction acts as a momentum sink (resistance to flow)
- Driving stress from surface slope provides primary forcing
- Effective pressure $N$ modulates basal resistance
"""))


### Momentum Balance Equation

**Strong form (force balance):**

$$-\nabla \cdot \boldsymbol{\sigma} - \rho_i g \nabla S = -\beta^2 N u_{\text{basal}} \quad \text{(at base)}$$

**Weak Form:**

The variational form integrates stress divergence over the domain:

$$\int_\Omega \left[-\frac{\partial \sigma_{xx}}{\partial x} - \rho g H \frac{\partial S}{\partial x}\right]\phi \, dx + \text{(stress boundary terms)} = \int_\Omega \beta^2 N u \phi \, dx$$

### Integrated Stress Components

Using the vertical basis function expansion, stresses are integrated:

$$\sigma_{xx}^{\text{integrated}} = \int_0^1 4\eta_v(s)H\left(\frac{\partial u}{\partial x}\bigg|_s\right) ds$$

$$\tau_{xz}^{\text{integrated}} = \int_0^1 \eta_v(s)H \left(\frac{ds}{dz}\right)^2 \frac{\partial u}{\partial s} ds$$

### Driving Stress (Nonstandard Formulation)

$$-\int_\Omega \frac{\partial(\rho g H \phi_{\bar{u}})}{\partial x}S \, dx + \text{(discontinuity corrections at element interfaces)} + \text{(boundary flux)}$$

**Physical Interpretation:**
- Momentum is transmitted through internal stresses
- Basal traction acts as a momentum sink (resistance to flow)
- Driving stress from surface slope provides primary forcing
- Effective pressure $N$ modulates basal resistance


## Section 9: Melt Rate and Water Production

Frictional heating at the glacier base produces meltwater.

In [37]:
display(Markdown(r"""
### Basal Melt Rate

**Heat balance (friction-driven melting):**

$$\dot{m}_e = \frac{\beta^2 N u_{\text{basal}}^2}{\rho_i L_a} - \min(\dot{a}, -10^{-16}) \cdot \text{sigmoid}(H - h_{\text{lim}}-1)$$

where:
- First term: friction heating converted to melt
- Second term: geothermal contribution (set to zero in this model)
- $L_a = 3.35 \times 10^5$ J/kg is latent heat of melting
- Sigmoid function smoothly applies threshold to thin ice

**Sigmoid Function:**

$$\text{sigmoid}(z) = \frac{1}{1 + e^{-z}}$$

**Water Flux Conservation:**

The water flux $Q_w$ is determined from the continuity equation:

$$\frac{\partial Q_w}{\partial x} = \dot{m}_e$$

**Physical Process:**
1. Basal velocity creates friction at glacier-bed interface
2. Friction generates heat: $\dot{q} = \beta^2 N u^2$
3. Heat melts ice: $\dot{m}_e = \dot{q}/L_a$
4. Meltwater flows downglacier as subglacial discharge
"""))


### Basal Melt Rate

**Heat balance (friction-driven melting):**

$$\dot{m}_e = \frac{\beta^2 N u_{\text{basal}}^2}{\rho_i L_a} - \min(\dot{a}, -10^{-16}) \cdot \text{sigmoid}(H - h_{\text{lim}}-1)$$

where:
- First term: friction heating converted to melt
- Second term: geothermal contribution (set to zero in this model)
- $L_a = 3.35 \times 10^5$ J/kg is latent heat of melting
- Sigmoid function smoothly applies threshold to thin ice

**Sigmoid Function:**

$$\text{sigmoid}(z) = \frac{1}{1 + e^{-z}}$$

**Water Flux Conservation:**

The water flux $Q_w$ is determined from the continuity equation:

$$\frac{\partial Q_w}{\partial x} = \dot{m}_e$$

**Physical Process:**
1. Basal velocity creates friction at glacier-bed interface
2. Friction generates heat: $\dot{q} = \beta^2 N u^2$
3. Heat melts ice: $\dot{m}_e = \dot{q}/L_a$
4. Meltwater flows downglacier as subglacial discharge


## Section 10: Sediment Transport - Erosion Processes

Three processes erode sediment: bedrock erosion, fluvial erosion, and deposition.

In [38]:
display(Markdown(r"""
### Bedrock Erosion

**Rate of bedrock lowering:**

$$\dot{B} = -b_e \beta^2 N u_{\text{basal}}^2 \delta$$

where:
- $b_e = 2 \times 10^{-9}$ yr$^{-1}$ is the erosion coefficient
- $\delta = e^{-h_s/l_s}$ is the mantling factor (reduced erosion under sediment)
- $l_s = 2$ m is the reference thickness for sediment mantling

**Mantling Effect:**

$$\delta = e^{-h_s/l_s}$$

**Physical Interpretation:**
- Erosion depends on normal stress $N$ and velocity squared $u^2$
- Stronger sediment cover reduces access to bedrock
- At $h_s = 0$: full erosion ($\delta = 1$)
- At $h_s \gg l_s$: erosion suppressed ($\delta \approx 0$)

### Fluvial Erosion (Subglacial)

**Detachment rate:**

$$\dot{e} = \frac{c_c}{h_{\text{eff}}} u_w^2 (1-\delta)$$

where:
- $c_c = 5 \times 10^{-11}$ yr$^{-1}$ is the fluvial erosion coefficient
- $h_{\text{eff}}$ is the effective subglacial cavity thickness
- $u_w = Q_w/h_{\text{eff}}$ is water velocity
- $(1-\delta)$ ensures erosion only where sediment cover is thin

**Physical Process:**
- Subglacial water flows over loose sediments
- Flow power $u_w^2$ entrains finer particles
- Effective only where bedrock is not mantled
- Scales inversely with water layer thickness (higher velocity in thin layers)
"""))


### Bedrock Erosion

**Rate of bedrock lowering:**

$$\dot{B} = -b_e \beta^2 N u_{\text{basal}}^2 \delta$$

where:
- $b_e = 2 \times 10^{-9}$ yr$^{-1}$ is the erosion coefficient
- $\delta = e^{-h_s/l_s}$ is the mantling factor (reduced erosion under sediment)
- $l_s = 2$ m is the reference thickness for sediment mantling

**Mantling Effect:**

$$\delta = e^{-h_s/l_s}$$

**Physical Interpretation:**
- Erosion depends on normal stress $N$ and velocity squared $u^2$
- Stronger sediment cover reduces access to bedrock
- At $h_s = 0$: full erosion ($\delta = 1$)
- At $h_s \gg l_s$: erosion suppressed ($\delta \approx 0$)

### Fluvial Erosion (Subglacial)

**Detachment rate:**

$$\dot{e} = \frac{c_c}{h_{\text{eff}}} u_w^2 (1-\delta)$$

where:
- $c_c = 5 \times 10^{-11}$ yr$^{-1}$ is the fluvial erosion coefficient
- $h_{\text{eff}}$ is the effective subglacial cavity thickness
- $u_w = Q_w/h_{\text{eff}}$ is water velocity
- $(1-\delta)$ ensures erosion only where sediment cover is thin

**Physical Process:**
- Subglacial water flows over loose sediments
- Flow power $u_w^2$ entrains finer particles
- Effective only where bedrock is not mantled
- Scales inversely with water layer thickness (higher velocity in thin layers)


## Section 11: Sediment Transport - Deposition and Settling

Suspended sediment is deposited through gravitational settling.

In [39]:
display(Markdown(r"""
### Deposition Rate

**Settling of suspended load:**

$$\dot{d} = \frac{d \cdot Q_s}{Q_w}$$

where:
- $d = 500$ is the fallout fraction (dimensionless coefficient)
- $Q_s$ is the fluvial sediment flux
- $Q_w$ is the water discharge

**Physical Interpretation:**
- Larger suspended sediment concentrations $Q_s/Q_w$ → more deposition
- Fallout fraction $d$ represents grain size and settling velocity effects
- As water slows (decreasing $Q_w$), deposition increases
- No deposition if no water flow ($Q_w = 0$)

### Sediment Flux Evolution

**Transport equation:**

$$\frac{\partial Q_s}{\partial t} + \frac{\partial}{\partial x}(Q_s u_w) = \dot{e} - \dot{d}$$

**Strong Form Interpretation:**
- Local sediment concentration changes with erosion/deposition
- Advection by water velocity transport sediment downglacier
- Net rate determined by erosion minus deposition

**Weak Form (Discontinuous Galerkin):**

$$\int_\Omega (\dot{e} - \dot{d})\psi_Q \, dx + \int_{\partial\Omega} Q_s^{\text{upwind}} \mathbf{n} \cdot \psi_Q \, ds = 0$$

where $Q_s^{\text{upwind}}$ uses upwind differencing for stability.
"""))


### Deposition Rate

**Settling of suspended load:**

$$\dot{d} = \frac{d \cdot Q_s}{Q_w}$$

where:
- $d = 500$ is the fallout fraction (dimensionless coefficient)
- $Q_s$ is the fluvial sediment flux
- $Q_w$ is the water discharge

**Physical Interpretation:**
- Larger suspended sediment concentrations $Q_s/Q_w$ → more deposition
- Fallout fraction $d$ represents grain size and settling velocity effects
- As water slows (decreasing $Q_w$), deposition increases
- No deposition if no water flow ($Q_w = 0$)

### Sediment Flux Evolution

**Transport equation:**

$$\frac{\partial Q_s}{\partial t} + \frac{\partial}{\partial x}(Q_s u_w) = \dot{e} - \dot{d}$$

**Strong Form Interpretation:**
- Local sediment concentration changes with erosion/deposition
- Advection by water velocity transport sediment downglacier
- Net rate determined by erosion minus deposition

**Weak Form (Discontinuous Galerkin):**

$$\int_\Omega (\dot{e} - \dot{d})\psi_Q \, dx + \int_{\partial\Omega} Q_s^{\text{upwind}} \mathbf{n} \cdot \psi_Q \, ds = 0$$

where $Q_s^{\text{upwind}}$ uses upwind differencing for stability.


## Section 12: Sediment Thickness Evolution

Conservation of sediment mass drives thickness changes.

In [40]:
display(Markdown(r"""
### Sediment Mass Balance

**Evolution of sediment layer:**

$$\frac{\partial h_s}{\partial t} = \frac{\rho_r}{\rho_s}\dot{B} - \dot{d} + \dot{e}$$

where:
- $\rho_r/\rho_s = 2650/1600 \approx 1.66$ is the density ratio
- $\dot{B}$ is bedrock erosion rate (negative, removes rock)
- $\dot{d}$ is deposition rate (positive, adds sediment)
- $\dot{e}$ is fluvial erosion rate (negative, removes sediment)

**Terms Explained:**

$$\text{Bedrock contribution: } \frac{\rho_r}{\rho_s}|\dot{B}| \text{ converts rock volume to sediment volume}$$

$$\text{Deposition adds mass: } +\dot{d}$$

$$\text{Fluvial erosion removes mass: } -\dot{e}$$

### Weak Form

$$\int_\Omega \psi_h \left[\frac{h_s - h_{s,0}}{\Delta t} + \frac{\rho_r}{\rho_s}\dot{B} - \dot{d} + \dot{e}\right] dx = 0$$

**Physical Process:**
1. Subglacial erosion removes bedrock; sediment thickness increases
2. Suspended load is transported downglacier
3. Settling deposition adds sediment in low-velocity regions
4. Fluvial erosion removes unconsolidated sediments
5. Net effect: sediment accumulates beneath glacier, forms moraines
"""))


### Sediment Mass Balance

**Evolution of sediment layer:**

$$\frac{\partial h_s}{\partial t} = \frac{\rho_r}{\rho_s}\dot{B} - \dot{d} + \dot{e}$$

where:
- $\rho_r/\rho_s = 2650/1600 \approx 1.66$ is the density ratio
- $\dot{B}$ is bedrock erosion rate (negative, removes rock)
- $\dot{d}$ is deposition rate (positive, adds sediment)
- $\dot{e}$ is fluvial erosion rate (negative, removes sediment)

**Terms Explained:**

$$\text{Bedrock contribution: } \frac{\rho_r}{\rho_s}|\dot{B}| \text{ converts rock volume to sediment volume}$$

$$\text{Deposition adds mass: } +\dot{d}$$

$$\text{Fluvial erosion removes mass: } -\dot{e}$$

### Weak Form

$$\int_\Omega \psi_h \left[\frac{h_s - h_{s,0}}{\Delta t} + \frac{\rho_r}{\rho_s}\dot{B} - \dot{d} + \dot{e}\right] dx = 0$$

**Physical Process:**
1. Subglacial erosion removes bedrock; sediment thickness increases
2. Suspended load is transported downglacier
3. Settling deposition adds sediment in low-velocity regions
4. Fluvial erosion removes unconsolidated sediments
5. Net effect: sediment accumulates beneath glacier, forms moraines


## Section 13: Bedrock Elevation Evolution

Bedrock is lowered by glacial erosion.

In [41]:
display(Markdown(r"""
### Bedrock Lowering

**Evolution equation:**

$$\frac{\partial B}{\partial t} = -\dot{B} = b_e \beta^2 N u_{\text{basal}}^2 \delta$$

or equivalently:

$$\frac{\partial B}{\partial t} = -b_e \beta^2 N u_{\text{basal}}^2 e^{-h_s/l_s}$$

**Weak Form:**

$$\int_\Omega \psi_B \left[\frac{B - B_0}{\Delta t} - \dot{B}\right] dx = 0$$

**Erosion Dependence:**

| Factor | Effect | Physical Reason |
|--------|--------|-----------------|
| $\uparrow N$ | $\uparrow$ erosion | Higher pressure → stronger abrasion |
| $\uparrow u$ | $\uparrow$ erosion (∝ $u^2$) | Faster motion → more contact |
| $\uparrow h_s$ | $\downarrow$ erosion (∝ $e^{-h_s/l_s}$) | Sediment cover protects bedrock |
| Grounded | $\uparrow$ erosion | Full coupling to glacier motion |
| Floating | $= 0$ | No effective pressure, no erosion |

**Characteristic Erosion Rate:**
- At high shear stress: $|\dot{B}| \sim 10^{-3}$ m/yr
- Under sediment cover: $|\dot{B}| \sim 10^{-6}$ m/yr (strongly reduced)
- Over model time 12,950 yr: possible 10-50 m of erosion under glacier
"""))


### Bedrock Lowering

**Evolution equation:**

$$\frac{\partial B}{\partial t} = -\dot{B} = b_e \beta^2 N u_{\text{basal}}^2 \delta$$

or equivalently:

$$\frac{\partial B}{\partial t} = -b_e \beta^2 N u_{\text{basal}}^2 e^{-h_s/l_s}$$

**Weak Form:**

$$\int_\Omega \psi_B \left[\frac{B - B_0}{\Delta t} - \dot{B}\right] dx = 0$$

**Erosion Dependence:**

| Factor | Effect | Physical Reason |
|--------|--------|-----------------|
| $\uparrow N$ | $\uparrow$ erosion | Higher pressure → stronger abrasion |
| $\uparrow u$ | $\uparrow$ erosion (∝ $u^2$) | Faster motion → more contact |
| $\uparrow h_s$ | $\downarrow$ erosion (∝ $e^{-h_s/l_s}$) | Sediment cover protects bedrock |
| Grounded | $\uparrow$ erosion | Full coupling to glacier motion |
| Floating | $= 0$ | No effective pressure, no erosion |

**Characteristic Erosion Rate:**
- At high shear stress: $|\dot{B}| \sim 10^{-3}$ m/yr
- Under sediment cover: $|\dot{B}| \sim 10^{-6}$ m/yr (strongly reduced)
- Over model time 12,950 yr: possible 10-50 m of erosion under glacier


## Section 14: Effective Water Layer Thickness

The subglacial water layer thickness is computed from cavity closure.

In [42]:
display(Markdown(r"""
### Effective Cavity Height

**Relationship to ice base and sediment:**

$$h_{\text{eff}} = \text{softplus}(h_0, S_{\text{base}} - B_{\text{hat}}, \alpha=5)$$

where:
- $h_0 = 0.1$ m is the background subglacial cavity depth
- $S_{\text{base}} - B_{\text{hat}}$ is the gap between ice base and sediment surface
- Softplus ensures smooth transition and differentiability

**Physical Interpretation:**

- **Thick sediment:** Gap smaller → $h_{\text{eff}} \approx h_0$ (thin water layer)
- **Thin sediment:** Gap larger → $h_{\text{eff}} = S_{\text{base}} - B_{\text{hat}}$ (water fills available space)
- **Floating ice:** Negative gap possible; smoothing prevents singularities

### Water Velocity

**Subglacial discharge velocity:**

$$u_w = \frac{Q_w}{h_{\text{eff}}}$$

**Relationship to sediment transport:**

- Larger $h_{\text{eff}}$ → slower water → less erosion potential
- Thin $h_{\text{eff}}$ → faster water → enhanced erosion
- Water acts as intermediary between glacier dynamics and sediment transport
"""))


### Effective Cavity Height

**Relationship to ice base and sediment:**

$$h_{\text{eff}} = \text{softplus}(h_0, S_{\text{base}} - B_{\text{hat}}, \alpha=5)$$

where:
- $h_0 = 0.1$ m is the background subglacial cavity depth
- $S_{\text{base}} - B_{\text{hat}}$ is the gap between ice base and sediment surface
- Softplus ensures smooth transition and differentiability

**Physical Interpretation:**

- **Thick sediment:** Gap smaller → $h_{\text{eff}} \approx h_0$ (thin water layer)
- **Thin sediment:** Gap larger → $h_{\text{eff}} = S_{\text{base}} - B_{\text{hat}}$ (water fills available space)
- **Floating ice:** Negative gap possible; smoothing prevents singularities

### Water Velocity

**Subglacial discharge velocity:**

$$u_w = \frac{Q_w}{h_{\text{eff}}}$$

**Relationship to sediment transport:**

- Larger $h_{\text{eff}}$ → slower water → less erosion potential
- Thin $h_{\text{eff}}$ → faster water → enhanced erosion
- Water acts as intermediary between glacier dynamics and sediment transport


## Section 15: Sediment Diffusion

Hillslope processes redistribute sediment laterally.

In [43]:
display(Markdown(r"""
### Sediment Diffusive Transport

**Hillslope flux (sediment diffusivity model):**

$$q_d = -k_{\text{diff}} f(h_s) \frac{\partial(B_{\text{hat}})}{\partial x}$$

where:
- $k_{\text{diff}} = 20$ m$^2$/yr is the sediment diffusivity
- $f(h_s) = 1 - e^{-h_s/h_{\text{ref}}}$ is the thickness-dependent efficiency
- $\nabla(B_{\text{hat}}) = \nabla(B + h_s)$ is the surface gradient

**Thickness-Dependent Efficiency:**

$$f(h_s) = 1 - e^{-h_s/h_{\text{ref}}}$$

**Behavior:**

| Thickness | $f(h_s)$ | Effect |
|-----------|----------|--------|
| $h_s \ll h_{\text{ref}}$ | $\approx h_s/h_{\text{ref}}$ | Flux proportional to thickness (thin moraines diffuse slowly) |
| $h_s \sim h_{\text{ref}}$ | $\approx 0.6$ | Transitional regime |
| $h_s \gg h_{\text{ref}}$ | $\approx 1$ | Flux thickness-independent (thick moraines diffuse at steady rate) |

**Physical Justification:**
- Thin sediment (weak moraines): slow redistribution
- Thick sediment (strong moraines): faster redistribution
- Avoids unphysical acceleration of thick deposits

### Diffusion Evolution Equation

**Sediment thickness change from diffusion:**

$$\frac{\partial h_s}{\partial t}\bigg|_{\text{diff}} = -\frac{\partial q_d}{\partial x} = k_{\text{diff}}\frac{\partial}{\partial x}\left[f(h_s)\frac{\partial B_{\text{hat}}}{\partial x}\right]$$

**Weak Form:**

$$\int_\Omega w_d\frac{h_s - h_{s,0}}{\Delta t} dx + \int_\Omega \tau_d q_d \, dx = \int_\Omega k_{\text{diff}}\frac{\partial \tau_d}{\partial x} f(h_s) \frac{\partial B_{\text{hat}}}{\partial x} dx$$
"""))


### Sediment Diffusive Transport

**Hillslope flux (sediment diffusivity model):**

$$q_d = -k_{\text{diff}} f(h_s) \frac{\partial(B_{\text{hat}})}{\partial x}$$

where:
- $k_{\text{diff}} = 20$ m$^2$/yr is the sediment diffusivity
- $f(h_s) = 1 - e^{-h_s/h_{\text{ref}}}$ is the thickness-dependent efficiency
- $\nabla(B_{\text{hat}}) = \nabla(B + h_s)$ is the surface gradient

**Thickness-Dependent Efficiency:**

$$f(h_s) = 1 - e^{-h_s/h_{\text{ref}}}$$

**Behavior:**

| Thickness | $f(h_s)$ | Effect |
|-----------|----------|--------|
| $h_s \ll h_{\text{ref}}$ | $\approx h_s/h_{\text{ref}}$ | Flux proportional to thickness (thin moraines diffuse slowly) |
| $h_s \sim h_{\text{ref}}$ | $\approx 0.6$ | Transitional regime |
| $h_s \gg h_{\text{ref}}$ | $\approx 1$ | Flux thickness-independent (thick moraines diffuse at steady rate) |

**Physical Justification:**
- Thin sediment (weak moraines): slow redistribution
- Thick sediment (strong moraines): faster redistribution
- Avoids unphysical acceleration of thick deposits

### Diffusion Evolution Equation

**Sediment thickness change from diffusion:**

$$\frac{\partial h_s}{\partial t}\bigg|_{\text{diff}} = -\frac{\partial q_d}{\partial x} = k_{\text{diff}}\frac{\partial}{\partial x}\left[f(h_s)\frac{\partial B_{\text{hat}}}{\partial x}\right]$$

**Weak Form:**

$$\int_\Omega w_d\frac{h_s - h_{s,0}}{\Delta t} dx + \int_\Omega \tau_d q_d \, dx = \int_\Omega k_{\text{diff}}\frac{\partial \tau_d}{\partial x} f(h_s) \frac{\partial B_{\text{hat}}}{\partial x} dx$$


## Section 16: Complete System Summary

Here is the coupled system of equations solved in sequence.

In [44]:
display(Markdown(r"""
### Coupled Evolution System

**Step 1: Velocity and Thickness (Nonlinear)**

Given $H^n, B^n, h_s^n$, solve for $\bar{u}^{n+1}, u_{\text{def}}^{n+1}, H^{n+1}$:

$$R_{\text{momentum}} + R_{\text{mass}} = 0$$

$$\text{constraints: } H \geq h_{\text{lim}}, \quad u \text{ bounded}$$

**Step 2: Water Flux (Linear)**

Solve for $Q_w^{n+1}$:

$$\frac{\partial Q_w}{\partial x} = \dot{m}_e(u^{n+1}, H^{n+1}, N^{n+1})$$

**Step 3: Sediment Transport (Nonlinear)**

Given $Q_w^{n+1}, N^{n+1}$, solve for $B^{n+1}, Q_s^{n+1}, h_s^{n+1}, h_{\text{eff}}^{n+1}$:

$$R_B + R_{Q_s} + R_{h_s} + R_{h_{\text{eff}}} = 0$$

**Step 4: Sediment Diffusion (Linear/Nonlinear)**

Solve for $h_s^{n+1,*}$ including hillslope processes:

$$\frac{\partial h_s}{\partial t} = -\frac{\partial}{\partial x}\left[k_{\text{diff}} f(h_s) \frac{\partial(B + h_s)}{\partial x}\right]$$

### Discretization

**Spatial:** Discontinuous Galerkin (DG) finite elements with 900 cells
- DG0: thickness, bedrock, sediment (piecewise constant)
- CG1: velocity, water pressure (continuous linear)

**Temporal:** Implicit Euler with adaptive time-stepping
- Initial: $\Delta t = 0.1$ yr
- Growth factor: $1.05 \times$
- Maximum: $\Delta t_{\max} = 1.0$ yr
- Reduction on convergence failure: $\Delta t \to \Delta t/2$

**Solver:** PETSc SNES with:
- VI-Newton-RSLS (bound-constrained)
- MUMPS direct linear algebra
- Relative tolerance: $10^{-4}$
- Absolute tolerance: $10^{-3}$
"""))


### Coupled Evolution System

**Step 1: Velocity and Thickness (Nonlinear)**

Given $H^n, B^n, h_s^n$, solve for $\bar{u}^{n+1}, u_{\text{def}}^{n+1}, H^{n+1}$:

$$R_{\text{momentum}} + R_{\text{mass}} = 0$$

$$\text{constraints: } H \geq h_{\text{lim}}, \quad u \text{ bounded}$$

**Step 2: Water Flux (Linear)**

Solve for $Q_w^{n+1}$:

$$\frac{\partial Q_w}{\partial x} = \dot{m}_e(u^{n+1}, H^{n+1}, N^{n+1})$$

**Step 3: Sediment Transport (Nonlinear)**

Given $Q_w^{n+1}, N^{n+1}$, solve for $B^{n+1}, Q_s^{n+1}, h_s^{n+1}, h_{\text{eff}}^{n+1}$:

$$R_B + R_{Q_s} + R_{h_s} + R_{h_{\text{eff}}} = 0$$

**Step 4: Sediment Diffusion (Linear/Nonlinear)**

Solve for $h_s^{n+1,*}$ including hillslope processes:

$$\frac{\partial h_s}{\partial t} = -\frac{\partial}{\partial x}\left[k_{\text{diff}} f(h_s) \frac{\partial(B + h_s)}{\partial x}\right]$$

### Discretization

**Spatial:** Discontinuous Galerkin (DG) finite elements with 900 cells
- DG0: thickness, bedrock, sediment (piecewise constant)
- CG1: velocity, water pressure (continuous linear)

**Temporal:** Implicit Euler with adaptive time-stepping
- Initial: $\Delta t = 0.1$ yr
- Growth factor: $1.05 \times$
- Maximum: $\Delta t_{\max} = 1.0$ yr
- Reduction on convergence failure: $\Delta t \to \Delta t/2$

**Solver:** PETSc SNES with:
- VI-Newton-RSLS (bound-constrained)
- MUMPS direct linear algebra
- Relative tolerance: $10^{-4}$
- Absolute tolerance: $10^{-3}$


## Section 17: Summary Table - Complete Variable Glossary

In [45]:
# Create comprehensive variable glossary
glossary = {
    "Primary Solution Variables": {
        r"$H$": ("m", "Ice thickness"),
        r"$\bar{u}$": ("m/yr", "Depth-averaged velocity"),
        r"$u_{\text{def}}$": ("m/yr", "Deformational velocity component"),
        r"$B$": ("m", "Bedrock elevation"),
        r"$h_s$": ("m", "Sediment layer thickness"),
        r"$Q_s$": ("m$^3$/yr", "Fluvial sediment flux"),
        r"$Q_w$": ("m$^3$/yr", "Subglacial water discharge"),
        r"$h_{\text{eff}}$": ("m", "Effective subglacial cavity thickness"),
    },
    "Derived Quantities": {
        r"$B_{\text{hat}}$": ("m", "Sediment surface elevation = $B + h_s$"),
        r"$\ell$": ("m", "Water surface elevation"),
        r"$S_{\text{base}}$": ("m", "Ice base elevation"),
        r"$S$": ("m", "Ice surface elevation = $S_{\text{base}} + H$"),
        r"$D$": ("m", "Water depth above sea level"),
        r"$P_0$": ("Pa", "Overburden pressure = $\\rho_i g H$"),
        r"$P_w$": ("Pa", "Water pressure"),
        r"$N$": ("Pa", "Effective pressure = $P_0 - P_w$"),
    },
    "Velocity Profile": {
        r"$u(s)$": ("m/yr", "Velocity at depth $s$"),
        r"$u(0)$": ("m/yr", "Basal velocity (at $s=0$)"),
        r"$u(1)$": ("m/yr", "Surface velocity (at $s=1$)"),
        r"$s$": ("-", "Normalized vertical coordinate"),
        r"$\phi_i(s)$": ("-", "Basis functions for depth-variation"),
    },
    "Stresses": {
        r"$\sigma_{xx}$": ("Pa", "Longitudinal (extensional) stress"),
        r"$\tau_{xz}$": ("Pa", "Vertical shear stress"),
        r"$\tau_b$": ("Pa", "Basal traction = $\\beta^2 u N$"),
        r"$\tau_d$": ("Pa", "Driving stress from surface slope"),
        r"$\eta_v$": ("Pa·yr", "Effective viscosity"),
    },
    "Erosion Rates": {
        r"$\dot{B}$": ("m/yr", "Bedrock erosion rate (lowering)"),
        r"$\dot{e}$": ("m/yr", "Fluvial erosion rate"),
        r"$\dot{d}$": ("m/yr", "Sediment deposition rate"),
        r"$\dot{m}_e$": ("m/yr", "Basal melt rate from friction"),
        r"$\delta$": ("-", "Mantling factor = $e^{-h_s/l_s}$"),
    },
    "Water/Sediment Transport": {
        r"$u_w$": ("m/yr", "Subglacial water velocity = $Q_w/h_{\text{eff}}$"),
        r"$q_d$": ("m$^2$/yr", "Sediment diffusive flux"),
        r"$f(h_s)$": ("-", "Thickness efficiency = $1 - e^{-h_s/h_\\text{ref}}$"),
    },
    "Mass Balance Terms": {
        r"$\dot{a}$": ("m/yr", "Surface mass balance (accumulation)"),
        r"$\mathcal{F}$": ("-", "Flotation indicator (0=grounded, 1=floating)"),
        r"$\text{scf}$": ("yr$^{-1}$", "Shelf collapse factor"),
    }
}

# Print glossary in organized sections
for section, variables in glossary.items():
    display(Markdown(f"### {section}\n"))
    md_table = "| Symbol | Units | Definition |\n|--------|-------|------------|\n"
    for symbol, (units, definition) in variables.items():
        md_table += f"| {symbol} | {units} | {definition} |\n"
    display(Markdown(md_table))

### Primary Solution Variables


| Symbol | Units | Definition |
|--------|-------|------------|
| $H$ | m | Ice thickness |
| $\bar{u}$ | m/yr | Depth-averaged velocity |
| $u_{\text{def}}$ | m/yr | Deformational velocity component |
| $B$ | m | Bedrock elevation |
| $h_s$ | m | Sediment layer thickness |
| $Q_s$ | m$^3$/yr | Fluvial sediment flux |
| $Q_w$ | m$^3$/yr | Subglacial water discharge |
| $h_{\text{eff}}$ | m | Effective subglacial cavity thickness |


### Derived Quantities


| Symbol | Units | Definition |
|--------|-------|------------|
| $B_{\text{hat}}$ | m | Sediment surface elevation = $B + h_s$ |
| $\ell$ | m | Water surface elevation |
| $S_{\text{base}}$ | m | Ice base elevation |
| $S$ | m | Ice surface elevation = $S_{	ext{base}} + H$ |
| $D$ | m | Water depth above sea level |
| $P_0$ | Pa | Overburden pressure = $\rho_i g H$ |
| $P_w$ | Pa | Water pressure |
| $N$ | Pa | Effective pressure = $P_0 - P_w$ |


### Velocity Profile


...

| Symbol | Units | Definition |
|--------|-------|------------|
| $\dot{B}$ | m/yr | Bedrock erosion rate (lowering) |
| $\dot{e}$ | m/yr | Fluvial erosion rate |
| $\dot{d}$ | m/yr | Sediment deposition rate |
| $\dot{m}_e$ | m/yr | Basal melt rate from friction |
| $\delta$ | - | Mantling factor = $e^{-h_s/l_s}$ |


### Water/Sediment Transport


| Symbol | Units | Definition |
|--------|-------|------------|
| $u_w$ | m/yr | Subglacial water velocity = $Q_w/h_{	ext{eff}}$ |
| $q_d$ | m$^2$/yr | Sediment diffusive flux |
| $f(h_s)$ | - | Thickness efficiency = $1 - e^{-h_s/h_\text{ref}}$ |


### Mass Balance Terms


| Symbol | Units | Definition |
|--------|-------|------------|
| $\dot{a}$ | m/yr | Surface mass balance (accumulation) |
| $\mathcal{F}$ | - | Flotation indicator (0=grounded, 1=floating) |
| $\text{scf}$ | yr$^{-1}$ | Shelf collapse factor |
